# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [12]:
# Import the zip bank data. Use dtype="object" to ensure all data is read in accurately.
weather_data_df = pd.read_csv(
    "../output_data/weather_data.csv", dtype="object", encoding="utf-8")

# Visualize
weather_data_df
weather_data_df.dropna(inplace=True)
weather_data_df

,Unnamed: 0,City,Lat,Lng,Cloudiness,Country,Date,Humidity,Max Temperature,Wind Speed
0,0,Kapaa,22.08,-159.32,40,US,1593141726,69,82.4,13.87
1,1,Mar del Plata,-38.0,-57.56,5,AR,1593141666,65,42.01,6.93
2,2,Haverfordwest,51.8,-4.97,87,GB,1593141728,93,66.0,2.24
3,3,Ponta do Sol,32.67,-17.1,40,PT,1593141729,64,68.0,6.93
4,4,Palmer,42.16,-72.33,15,US,1593141730,64,70.0,2.06
...,...,...,...,...,...,...,...,...,...,...
571,571,Verkhovazh'ye,60.75,42.05,48,RU,1593142361,65,59.67,6.11
572,572,Muravlenko,63.79,74.52,81,RU,1593142362,86,51.31,16.08
573,573,Portage,41.17,-81.2,40,US,1593142364,87,66.0,6.31
574,574,Korla,41.76,86.15,42,CN,1593142365,80,69.01,10.98


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [14]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_data_df[["Lat", "Lng"]].astype(float)

# Convert Humidity to float and store
# HINT: be sure to handle NaN values
weather_data_df = weather_data_df.dropna()
humidity = weather_data_df["Humidity"].astype(float)
# weather_data_df

In [15]:
# Create a Heatmap layer

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
# Set filters on ideal vacation city

# convert max temp, wind speed and cloudiness to numbers
weather_data_df['Max Temperature'] = pd.to_numeric(weather_data_df['Max Temperature'])
weather_data_df['Wind Speed'] = pd.to_numeric(weather_data_df['Wind Speed'])
weather_data_df['Cloudiness'] = pd.to_numeric(weather_data_df['Cloudiness'])

# temp between 70-80
ideal_temp = (weather_data_df['Max Temperature'] > 70) & (weather_data_df['Max Temperature'] < 80)
vacation_city_temp = weather_data_df.loc[ideal_temp]

#Clouds less than 25
vacation_city_temp_winds = vacation_city_temp.loc[(vacation_city_temp['Wind Speed'] < 10)]

#humidity less than 60
vacation_city_temp_winds_clouds = vacation_city_temp_winds.loc[(vacation_city_temp_winds['Cloudiness'] == 0)]
vacation_city_temp_winds_clouds

,Unnamed: 0,City,Lat,Lng,Cloudiness,Country,Date,Humidity,Max Temperature,Wind Speed
13,13,Morondava,-20.28,44.28,0,MG,1593141740,90,70.88,8.34
81,81,Poya,-21.35,165.15,0,NC,1593141815,49,73.33,8.70
106,106,Troina,37.79,14.6,0,IT,1593141843,58,73.99,1.90
147,147,Najrān,17.49,44.13,0,SA,1593141831,22,75.20,2.24
166,166,Saint Lawrence,36.06,14.2,0,MT,1593141909,88,71.60,4.70
193,193,Nouakchott,18.09,-15.98,0,MR,1593141939,88,75.20,8.05
234,234,Lixouri,38.2,20.43,0,GR,1593141984,77,70.21,5.66
299,299,Bogotol,56.21,89.53,0,RU,1593142057,71,70.09,5.03
427,427,Alta Floresta,-9.88,-56.09,0,BR,1593142199,77,72.12,1.61
459,459,Malesina,38.62,23.23,0,GR,1593142235,64,75.99,8.99


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Set dataframe to hotel_df
hotel_df = vacation_city_temp_winds_clouds

hotel_df["Hotel Name"] =""

# Store 'Lat' and 'Lng' into  locations 
locations = hotel_df[["Lat", "Lng"]].astype(float)
hotel_df

C:\Users\chrcampb\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Unnamed: 0,City,Lat,Lng,Cloudiness,Country,Date,Humidity,Max Temperature,Wind Speed,Hotel Name
44,44,Kristinehamn,59.31,14.11,0,SE,1592943634,77,71.01,6.93,
70,70,Kigoma,-4.88,29.63,0,TZ,1592943663,78,71.60,3.29,
87,87,Tūkrah,32.53,20.58,0,LY,1592943682,73,70.95,6.15,
240,240,Mahajanga,-15.72,46.32,0,MG,1592943855,75,74.80,2.19,
243,243,Aktau,43.65,51.2,0,KZ,1592943858,35,73.40,8.95,
255,255,Steinbach am Taunus,50.17,8.57,0,DE,1592943872,36,77.00,8.05,
274,274,Al Bardīyah,31.76,25.09,0,LY,1592943893,80,70.48,8.12,
303,303,Nhulunbuy,-12.23,136.77,0,AU,1592943926,88,71.60,5.82,
311,311,Ilebo,-4.32,20.58,0,CD,1592943935,49,73.45,1.30,
341,341,Vao,-22.67,167.48,0,NC,1592943968,77,72.18,9.15,


In [19]:
# Put in locations and get Hotel Name

target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():

    # get location from hotel_df
    location = f"{row['Lat']}, {row['Lng']}"

    # add keyword to params dict
    params['location'] = location

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {location}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']

    try:
        print(f"Closest hotel to coordinates: {location} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


hotel_df

Retrieving Results for Index 44: 59.31, 14.11.
Closest hotel to coordinates: 59.31, 14.11 is Hotell Marieberg.
Retrieving Results for Index 70: -4.88, 29.63.
Closest hotel to coordinates: -4.88, 29.63 is Kigoma Hilltop Hotel.
Retrieving Results for Index 87: 32.53, 20.58.
Closest hotel to coordinates: 32.53, 20.58 is استراحة لاقروتا.
Retrieving Results for Index 240: -15.72, 46.32.
Closest hotel to coordinates: -15.72, 46.32 is Restaurant Greedy Coco Lodge Majunga.
Retrieving Results for Index 243: 43.65, 51.2.
Closest hotel to coordinates: 43.65, 51.2 is Silk Way Hotel.
Retrieving Results for Index 255: 50.17, 8.57.
Closest hotel to coordinates: 50.17, 8.57 is Mercure Hotel Frankfurt Eschborn Helfmann Park.
Retrieving Results for Index 274: 31.76, 25.09.
Closest hotel to coordinates: 31.76, 25.09 is بورتو بردية.
Retrieving Results for Index 303: -12.23, 136.77.
Closest hotel to coordinates: -12.23, 136.77 is East Arnhem Real Estate.
Retrieving Results for Index 311: -4.32, 20.58.
Clos

,Unnamed: 0,City,Lat,Lng,Cloudiness,Country,Date,Humidity,Max Temperature,Wind Speed,Hotel Name
44,44,Kristinehamn,59.31,14.11,0,SE,1592943634,77,71.01,6.93,Hotell Marieberg
70,70,Kigoma,-4.88,29.63,0,TZ,1592943663,78,71.60,3.29,Kigoma Hilltop Hotel
87,87,Tūkrah,32.53,20.58,0,LY,1592943682,73,70.95,6.15,استراحة لاقروتا
240,240,Mahajanga,-15.72,46.32,0,MG,1592943855,75,74.80,2.19,Restaurant Greedy Coco Lodge Majunga
243,243,Aktau,43.65,51.2,0,KZ,1592943858,35,73.40,8.95,Silk Way Hotel
255,255,Steinbach am Taunus,50.17,8.57,0,DE,1592943872,36,77.00,8.05,Mercure Hotel Frankfurt Eschborn Helfmann Park
274,274,Al Bardīyah,31.76,25.09,0,LY,1592943893,80,70.48,8.12,بورتو بردية
303,303,Nhulunbuy,-12.23,136.77,0,AU,1592943926,88,71.60,5.82,East Arnhem Real Estate
311,311,Ilebo,-4.32,20.58,0,CD,1592943935,49,73.45,1.30,Hôtel des palmes
341,341,Vao,-22.67,167.48,0,NC,1592943968,77,72.18,9.15,Hotel Kou-Bugny


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [17]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display figure
fig


IndexError: list index out of range